In [14]:
import rasterio
from rasterio.io import MemoryFile
from rasterio.warp import calculate_default_transform
from rasterio.warp import reproject
import numpy as np

In [5]:
file_path = 'data/rvrnbrt.tif'
src_dataset = rasterio.open(file_path)
dst_crs = "EPSG:4326"

transform, width, height = calculate_default_transform(
    src_dataset.crs,
    dst_crs,
    src_dataset.width,
    src_dataset.height,
    *src_dataset.bounds
)

dst_bands = []
for band_n_1 in range(src_dataset.count):
    src_band = rasterio.band(src_dataset, band_n_1 + 1)
    dst_band = reproject(src_band, dst_crs=dst_crs)
    dst_bands.append(dst_band)

(bounds_bottom, bounds_right) = transform * (0, 0)
(bounds_top, bounds_left) = transform * (width, height)
bounds = [[bounds_left, bounds_bottom], [bounds_right, bounds_top]]

In [13]:
type(dst_bands[0][0])

numpy.ndarray

In [15]:
arr = np.atleast_3d(dst_bands[0][0])
height, width, nblayers = arr.shape

In [22]:
print(dst_bands[0][0][0].shape)

(2290, 4511)


In [20]:
image = np.zeros((61, 61))
image[0, :] = 1.0
image[60, :] = 1.0
image[:, 0] = 1.0
image[:, 60] = 1.0

In [21]:
image.shape

(61, 61)

In [ ]:
src_dataset = rasterio.open(file_path)
dst_crs = "EPSG:4326"

transform, width, height = calculate_default_transform(
    src_dataset.crs,
    dst_crs,
    src_dataset.width,
    src_dataset.height,
    *src_dataset.bounds
)

dst_bands = []
for band_n_1 in range(src_dataset.count):
    src_band = rasterio.band(src_dataset, band_n_1 + 1)
    dst_band = reproject(src_band, dst_crs=dst_crs)
    dst_bands.append(dst_band)

if src_dataset.count != 3:
    for i in range(len(dst_bands), src_dataset.count):
        dst_bands.append(rasterio.band(src_dataset, 1))

alpha = np.where(dst_bands[0][0] > 1e8, 0, 1)
alpha_band = list(copy.deepcopy(dst_bands[0]))
alpha_band[0] = alpha.astype("uint8")
dst_bands.append(tuple(alpha_band))

png_kwargs = src_dataset.meta.copy()
png_kwargs.update(
    {
        "crs": dst_crs,
        "width": width,
        "height": height,
        "driver": "PNG",
        "dtype": rasterio.uint8,
        "transform": transform,
        "count": len(dst_bands),
    }
)

with MemoryFile() as png_memfile:
    with png_memfile.open(**png_kwargs) as dst_file:
        for i_1, dst_band in enumerate(dst_bands):
            dst_file.write(dst_band[0][0], i_1 + 1)

            dst_file.write_colormap(
                i_1 + 1, {0: (255, 0, 0, 255), 255: (0, 0, 0, 255)}
            )

        self.raster_image_reference.append(png_memfile.read())

    url = (
        "data:image/png;base64," +
        base64.b64encode(png_memfile.read()).decode()
    )
    (bounds_bottom, bounds_right) = transform * (0, 0)
    (bounds_top, bounds_left) = transform * (width, height)
    bounds = [[bounds_left, bounds_bottom], [bounds_right, bounds_top]]
